<a href="https://colab.research.google.com/github/Rahulroy65/AgenticAI/blob/main/Travel_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📝 Test Case: Planning a Last-Minute Cruise Trip to Hawai Islands .

Scenario:

You have only 7 days for the trip.

Your budget is $5000 (including flights, stay, food, transport).

You prefer beaches over shopping.

You need to find a hotel near public transport.

You want a real-time weather forecast for Hawaii before finalizing the plan.


Step1 : Install packages for crewai and crewaitools

!pip install crewai

!pip install crewai-tools

In [12]:
!pip install crewai
!pip install crewai-tools

ackages (from chromadb>=0.5.23->crewai) (3.10.18)
lready satisfied: langchain-openai<0.3.0,>=0.2.1 in /usr/local/lib/python3.11/dist-packages (from embedchain>=0.1.114->crewai-tools) (0.2.14)


Step2 : Import Packages

In [61]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

Step3 : Configure Google Gemini and Serper API using their API Key

In [62]:
#If you do not have API key, Please register from GoogleAI studio and Serper OpenAI websites.
from google.colab import userdata
gemini_key=userdata.get('gemini_key')
serper_key=userdata.get('serper_key')
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

Step4 : Define LLM model and Intialize SeperDevTool

In [63]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

Step5 : Build your Agents

In [64]:
# 🧭 Travel Researcher Agent (Finds beaches+ hotels near public transport + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find beaches, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about Scenic trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True
)

# 💰 Budget Planner Agent (Ensures trip stays under $5000)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget cruise, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 2-day itinerary for {destination}, ensuring all beaches are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)


Step6: Define Tasks for this Agents

In [65]:
# 📝 Travel Research Task
research_task = Task(
    description="Find the best beaches, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top  beaches, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flight, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (cruise, hotel, food, attractions) ensuring a $5000 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 2-day itinerary for {destination}, focusing on beaches, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 2-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)


Step7: Create Crew with agents and their tasks

In [66]:

# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for Hawaii with a $5000 budget
result = crew.kickoff(inputs={'destination': 'Hawaii', 'budget': '5000'})
print(result)


# Agent: Travel Researcher
## Task: Find the best beaches, weather forecast, and public transport hotels for Hawaii.


# Agent: Travel Researcher
## Thought: tool_code
Thought: I need to gather information about beaches, weather, and hotels near public transport in Hawaii using the available tools.  I'll start by searching for beaches and then look up weather and hotels.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"best beaches in Hawaii\"}"
## Tool Output: 
{'searchParameters': {'q': 'best beaches in Hawaii', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "HAWAII'S BEST BEACHES | Hannah Rose Gray", 'link': 'https://hannahrosegray.com/blog/hawaiis-best-beaches/', 'snippet': "HAWAII'S BEST BEACHES · 2. Waianapanapa Black Sand Beach – Maui · 3. Ke'e Beach – Kauai · 4. Baby Beach, Sprecklesville Area – Maui · 5. Kua Bay ...", 'position': 1}, {'title': 'Trip Planning: What are the best beaches to see in Hawaii? - Reddit', 'li